In [1]:
# this notebook generates all commands for the recent mml version 
import dataclasses
import os
import warnings
from pathlib import Path
from typing import Dict, Union

try:
    import mml.interactive
except ImportError:
    raise RuntimeError('This reproduction expects a recent version of MML - please refer to the README for detailed instructions.')

mml.interactive.init()
from mml.interactive import DefaultRequirements, MMLJobDescription
from mml_tf.tasks import all_tasks, get_valid_sources, shrinkable_tasks, target_tasks, source_tasks, train_tasks, \
    all_tasks_including_shrunk

CLUSTER_USAGE = False  # change if you (want / do not want) to run on the cluster

if CLUSTER_USAGE:
    from mml_lsf.requirements import LSFSubmissionRequirements

/home/scholzpa/Documents/development/gitlab/mml/src/mml/interactive/__init__.py:49: UserWarning: Did not provide a "env_path", neither found set "MML_ENV_PATH" variable, you might need to provide "env_path" to "init" in order to use "mml" interactively in a jupyter/ipython setting.
  warnings.warn(
/home/scholzpa/miniconda3/envs/mml/lib/python3.8/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


 _____ ______   _____ ______   ___
|\   _ \  _   \|\   _ \  _   \|\  \
\ \  \\\__\ \  \ \  \\\__\ \  \ \  \
 \ \  \\|__| \  \ \  \\|__| \  \ \  \
  \ \  \    \ \  \ \  \    \ \  \ \  \____
   \ \__\    \ \__\ \__\    \ \__\ \_______\
    \|__|     \|__|\|__|     \|__|\|_______|
         ____  _  _    __  _  _  ____  _  _
        (  _ \( \/ )  (  )( \/ )/ ___)( \/ )
         ) _ ( )  /    )( / \/ \\___ \ )  /
        (____/(__/    (__)\_)(_/(____/(__/
Interactive MML API initialized.


In [3]:
task_infos = mml.interactive.get_task_infos(task_list=all_tasks, dims=None)

In [4]:
# convenience function for easier retrieve from cluster results
user_id = 'USERNAME'
# note that final experiments have to be run multiple times to ensure significance
rerun = 3


# use as print(get_retrieve_for_proj('my_project')) and run the result in a shell to get the results of 'my_project' from cluster to your local system
def get_retrieve_for_proj(proj):
    return f"rsync -rtvu --stats --exclude=PARAMETERS --exclude=hpo --exclude=runs --exclude=FIMS --exclude=FC_TUNED {user_id}@odcf-worker01:{os.getenv('MML_CLUSTER_RESULTS_PATH')}/{proj}/ {os.getenv('MML_RESULTS_PATH')}/{proj}"


# the following optimizes a jobs epochs in a way that target task is seen at least 40 epochs but at max 4000 steps (plus finishing the epoch)
def optimize_epochs(target_task: str, batch_size: int = 300, max_steps: int = 4000, max_epoch: int = 40) -> int:
    return min(max_epoch, (max_steps // ((int(task_infos.num_samples[target_task] * 0.8) // batch_size) + 1)) + 1)

In [5]:
if CLUSTER_USAGE:
    # cluster submission prepends, add yours here in case you have other gpu requirements
    base_reqs = LSFSubmissionRequirements(special_requirements=[],
                                          undesired_hosts=['e230-dgx2-2', 'e230-dgxa100-2', 'e230-dgxa100-4',
                                                           'e230-dgxa100-1',
                                                           'e230-dgxa100-2', 'e230-dgxa100-3', 'e230-dgxa100-4',
                                                           'lsf22-gpu08', 'lsf22-gpu01', 'lsf22-gpu02', 'lsf22-gpu03',
                                                           'lsf22-gpu04', 'lsf22-gpu05', 'lsf22-gpu06', 'lsf22-gpu07'],
                                          num_gpus=1, vram_per_gpu=11.0, queue='gpu-lowprio',
                                          mail='EMAIL.ADDRESS@dkfz-heidelberg.de', script_name='mml.sh',
                                          job_group='/USERNAME/pami_rerun'
                                          )
    # see for example this local setup
    # base_reqs = pp_reqs = aa_reqs = def_reqs = arch_reqs = tl_reqs = multi_reqs = nb.DefaultRequirements()
    pp_reqs = dataclasses.replace(base_reqs, queue='gpu')
    aa_reqs = dataclasses.replace(base_reqs, script_name='aa.sh', vram_per_gpu=13.0)
    def_reqs = dataclasses.replace(base_reqs, special_requirements=['tensorcore'])
    tl_reqs = dataclasses.replace(base_reqs, special_requirements=['tensorcore'], vram_per_gpu=24.0)
    multi_reqs = dataclasses.replace(base_reqs, special_requirements=['tensorcore'], vram_per_gpu=14.0)
else:
    base_reqs = pp_reqs = aa_reqs = def_reqs = tl_reqs = multi_reqs = DefaultRequirements()

In [6]:
# project overview -> points to MML projects we use
projects = {
    'base': 'pami2_base_02',
    'fed_hpo': 'pami2_fed_hpo_10',
    'raw_baseline': 'pami2_raw_03',
    'raw_shrunk': 'pami2_raw_shrunk_10',
    # aa search was misplaced by accident, this is resolved within exp 3
    'aa_search': 'pami2_t_aa_search_01',
    # the above are shared with train (!) since stuff is only computed once anyway
    'pretrain': 'pami2_t_pretrain_01',
    'transfer': 'pami2_t_transfer_20',
    'multi_task': 'test_multi_balanced_test_split_10',  # 'pami2_t_multi_task_02',
    'multi_shrunk': 'test_multi_balanced_shrunk_test_split_10',  # 'pami2_t_multi_shrunk_02',
    'arch_search': 'pami2_t_arch_search_02',
    'arch_infer': 'pami2_t_arch_infer_02',
    'aa_infer': 'pami2_t_aa_infer_02'
}

In [7]:
# prepare steps
prep_cmds = list()
# step one: plain task creation

prep_cmds.append(MMLJobDescription(prefix_req=pp_reqs, mode='create',
                                   config_options={'tasks': 'pami', 'proj': projects['base']}))
# step two: plain task preprocessing
prep_cmds.append(MMLJobDescription(prefix_req=pp_reqs, mode='pp',
                                   config_options={'tasks': 'pami', 'proj': projects['base']}))
# step three: shrunk preprocessing
prep_cmds.append(MMLJobDescription(prefix_req=pp_reqs, mode='info',
                                   config_options={'tasks': 'pami_shrinkable_800', 'proj': projects['base']}))
mml.interactive.write_out_commands(cmd_list=prep_cmds, name='prepare')

Stored 3 commands at prepare.txt.


In [8]:
# OPTIONALLY: compute dimensions (used for Fig. 3) and some additional experiments not shown in the paper
dim_cmds = list()
dim_cmds.append(MMLJobDescription(prefix_req=def_reqs, mode='dim', config_options={'tasks': 'pami_shrink_mix',
                                                                                   'proj': projects["base"],
                                                                                   'mode.inv_mle': True}))
mml.interactive.write_out_commands(cmd_list=dim_cmds, name='dimensions')

Stored 1 commands at dimensions.txt.


In [9]:
# baselines
# these are the default options for all tasks, they should not be modified without justification
def get_default_config(target_task: str, shrunk: bool = False) -> Dict[str, Union[str, bool, int]]:
    if shrunk:
        epochs = 40
    else:
        epochs = optimize_epochs(target_task=target_task, batch_size=300, max_steps=4000, max_epoch=40)
    default_options = {'tasks': 'pami', 'pivot.name': t, 'mode.cv': False, 'mode.nested': False,
                       'mode.store_parameters': False, 'sampling.balanced': True,
                       'sampling.batch_size': 300, 'callbacks': 'none', 'lr_scheduler': 'step',
                       '+trainer.check_val_every_n_epoch': epochs,
                       'trainer.max_epochs': epochs, 'augmentations': 'baseline256', 'sampling.enable_caching': True}
    return default_options


base_cmds = list()
for ix in range(rerun):
    for t in all_tasks:
        opts = get_default_config(t)
        opts.update({'proj': f'{projects["raw_baseline"]}_{ix}', 'seed': ix, 'mode.store_parameters': True})
        base_cmds.append(MMLJobDescription(prefix_req=def_reqs, mode='train', config_options=opts))
        if t in shrinkable_tasks:
            shrink_opts = get_default_config(t, shrunk=True)
            shrink_opts.update({'proj': f'{projects["raw_shrunk"]}_{ix}', 'tasks': 'pami_shrink'})
            base_cmds.append(MMLJobDescription(prefix_req=def_reqs, mode='train', config_options=shrink_opts))
mml.interactive.write_out_commands(cmd_list=base_cmds, name='baseline')

Stored 381 commands at baseline.txt.


In [12]:
#################################
# EXPERIMENT 1: MODEL TRANSFER  #
#################################
# VRAM requirements for timm architectures
model_transfer_arch_reqs = {
    'tf_efficientnet_b2': 23.0,
    'tf_efficientnet_b2_ap': 24.0,
    'tf_efficientnet_b2_ns': 24.0,
    'tf_efficientnet_cc_b0_4e': 22.0,
    'swsl_resnet50': 20.0,
    'ssl_resnext50_32x4d': 24.0,
    'regnetx_032': 20.5,
    'regnety_032': 22.0,
    'rexnet_100': 20.5,
    'ecaresnet50d': 24.0,
    'cspdarknet53': 23.0,
    'mixnet_l': 25.0,
    'cspresnext50': 24.0,
    'cspresnet50': 18.0,
    'ese_vovnet39b': 25.0,
    'resnest50d': 25.5,
    'hrnet_w18': 24.0,
    'skresnet34': 16.5,
    'mobilenetv3_large_100': 13.5,
    'res2net50_26w_4s': 24.5
}
arch_cmds = list()
for ix in range(rerun):
    for t in source_tasks:
        for arch, vram in model_transfer_arch_reqs.items():
            opts = get_default_config(t)
            opts.update({'proj': f'{projects["arch_search"]}_{ix}',
                         'arch.timm.name': arch, 'seed': ix})
            # the following goes back to a rare occurrence of incompatible singleton batches with some batch_norms
            # avoid this by minimally wiggle batch size
            if t == 'mura_xr_wrist' and arch in ['rexnet_100', 'resnest50d', 'skresnet34']:
                opts.update({'sampling.batch_size': 301})
            if CLUSTER_USAGE:
                arch_reqs = dataclasses.replace(def_reqs, vram_per_gpu=vram)
            else:
                arch_reqs = def_reqs
            arch_cmds.append(MMLJobDescription(prefix_req=arch_reqs, mode='train',
                                               config_options=opts))
mml.interactive.write_out_commands(cmd_list=arch_cmds, name='full_arch', max_cmds=2000)
arch_shrunk_cmds = list()
for ix in range(rerun):
    for t in target_tasks:
        if task_infos.num_classes[t] > 40 or task_infos.num_samples[t] <= 1000:
            continue
        for arch, vram in model_transfer_arch_reqs.items():
            opts = get_default_config(t, shrunk=True)
            opts.update({'proj': f'{projects["arch_infer"]}_{ix}', 'tasks': 'pami_shrink',
                         'arch.classification.id': arch, 'seed': ix})
            if CLUSTER_USAGE:
                arch_reqs = dataclasses.replace(def_reqs, vram_per_gpu=vram)
            else:
                arch_reqs = def_reqs
            arch_shrunk_cmds.append(MMLJobDescription(prefix_req=arch_reqs, mode='train',
                                                         config_options=opts))
mml.interactive.write_out_commands(cmd_list=arch_shrunk_cmds, name='arch_shrunk', max_cmds=2000)

Stored 2000 commands at full_arch_0.txt.
Stored 2000 commands at full_arch_1.txt.
Stored 260 commands at full_arch_2.txt.
Stored 2000 commands at arch_shrunk_0.txt.
Stored 160 commands at arch_shrunk_1.txt.


In [13]:
####################################
# EXPERIMENT 2: TRANSFER LEARNING  #
####################################
trans_cmds = list()
for ix in range(rerun):
    for t in target_tasks:
        # only small tasks are used as targets
        if task_infos.num_classes[t] > 40:
            continue
        for s in get_valid_sources(t):
            mod_task_file = 'pami' if task_infos.num_samples[t] <= 1000 else 'pami_shrink'
            opts = get_default_config(t, shrunk=True)
            opts.update({'proj': f'{projects["transfer"]}_{ix}', 'tasks': mod_task_file,
                         'reuse.models': f'{projects["raw_baseline"]}_{ix}', 'mode.pretrain_task': s,
                         'seed': ix})
            trans_cmds.append(MMLJobDescription(prefix_req=def_reqs, config_options=opts, mode='tl'))
mml.interactive.write_out_commands(cmd_list=trans_cmds, name='transfer', max_cmds=2000)

Stored 2000 commands at transfer_0.txt.
Stored 2000 commands at transfer_1.txt.
Stored 2000 commands at transfer_2.txt.
Stored 2000 commands at transfer_3.txt.
Stored 496 commands at transfer_4.txt.


In [14]:
######################################
# EXPERIMENT 3: AUG POLICY TRANSFER  #
######################################
# Step 1:  training the auto augmentation pipeline for each potential source
warnings.warn("AA mode is not supported anymore with the recent version of MML, you need to import the following projects manually -> pami2_t_aa_search_01 and pami2_aa_search_01")
# Step 2: evaluating the augmentation pipeline
policy_cmds = list()
for ix in range(rerun):
    for t in target_tasks:
        # only small tasks are used as targets
        if task_infos.num_classes[t] > 40:
            continue
        for s in get_valid_sources(t):
            mod_task_file = 'pami' if task_infos.num_samples[t] <= 1000 else 'pami_shrink'
            opts = get_default_config(t, shrunk=True)
            # resolving an accident from above
            reuse_aa_proj = f'pami2_aa_search_01_{ix}' if s in train_tasks else f'pami2_t_aa_search_01_{ix}'
            # and another accident
            if 'breast' in s:
                reuse_aa_proj = f'pami2_t_aa_search_01_{ix}'
            opts.update({'proj': f'{projects["aa_infer"]}_{ix}', 'tasks': mod_task_file,
                         'reuse.aa': reuse_aa_proj, 'augmentations': 'load_aa_from',
                         'augmentations.source': s, 'seed': ix})
            policy_cmds.append(MMLJobDescription(prefix_req=def_reqs, config_options=opts, mode='train'))
mml.interactive.write_out_commands(cmd_list=policy_cmds, name='policy', max_cmds=2000)

Stored 2000 commands at policy_0.txt.
Stored 2000 commands at policy_1.txt.
Stored 2000 commands at policy_2.txt.
Stored 2000 commands at policy_3.txt.
Stored 496 commands at policy_4.txt.


/tmp/ipykernel_3261094/1949134110.py:5: UserWarning: AA mode is not supported anymore with the recent version of MML, you need to import the following projects manually -> pami2_t_aa_search_01 and pami2_aa_search_01
  warnings.warn("AA mode is not supported anymore with the recent version of MML, you need to import the following projects manually -> pami2_t_aa_search_01 and pami2_aa_search_01")


In [18]:
######################################
# EXPERIMENT 4: MULTI-TASK LEARNING  #
######################################
# We did not use full multitask learning with full sized target tasks in the paper
# multi_cmds = list()
# for ix in range(rerun):
#     for t in target_tasks:
#         for s in get_valid_sources(t):
#             opts = get_default_config(t)
#             opts.update(
#                 {
#                     'proj': f'{projects["multi_task"]}_{ix}',
#                     'mode.multitask': 2,
#                     'sampling.balanced': True,
#                     'mode.co_tasks': [s],
#                     'sampling.sample_num': int(0.8 * task_infos.num_samples[t]),
#                     'loss.auto_activate_weighing': False, 'seed': ix})
#             multi_cmds.append(MMLJobDescription(prefix_req=def_reqs, config_options=opts, mode='train'))
# mml.interactive.write_out_commands(cmd_list=multi_cmds, name='full_multi', max_cmds=2000)

multi_shrunk_cmds = list()
for ix in range(rerun):
    for t in target_tasks:
        # unshrinkable or already covered above
        if task_infos.num_classes[t] > 40 or task_infos.num_samples[t] <= 1000:
            continue
        for s in get_valid_sources(t):
            opts = get_default_config(t, shrunk=True)
            opts.update(
                {'tasks': 'pami_shrink',
                 'proj': f'{projects["multi_shrunk"]}_{ix}',
                 'mode.multitask': 2,
                 'sampling.balanced': True,
                 'mode.co_tasks': [s],
                 'sampling.sample_num': min(int(0.8 * task_infos.num_samples[t]), 800),
                 'loss.auto_activate_weighing': False, 'seed': ix})
            multi_shrunk_cmds.append(MMLJobDescription(prefix_req=def_reqs, config_options=opts, mode='train'))
mml.interactive.write_out_commands(cmd_list=multi_shrunk_cmds, name='multi_shrunk', max_cmds=2000)

Stored 2000 commands at multi_shrunk_0.txt.
Stored 2000 commands at multi_shrunk_1.txt.
Stored 2000 commands at multi_shrunk_2.txt.
Stored 1026 commands at multi_shrunk_3.txt.


In [19]:
all_train_cmds = base_cmds + arch_cmds + arch_shrunk_cmds + trans_cmds + policy_cmds + multi_shrunk_cmds
print(f'Our experiments trained {len(all_train_cmds)} models.')

Our experiments trained 30819 models.


In [20]:
# if you want to submit jobs to the cluster or run them locally, consider the runner functionality
# see mml_lsf README instructions on how to set this up 
# the following demonstrates submission of the baseline jobs
if CLUSTER_USAGE:
    from mml_lsf.runner import LSFJobRunner

    runner = LSFJobRunner()
    for job in base_cmds:
        runner.run(job)

In [22]:
sync_cmds = list()
for ix in range(rerun):
    for proj_id in ['multi_task', 'aa_infer', 'transfer', 'arch_search', 'raw_shrunk',
                    'raw_baseline', 'multi_shrunk', 'arch_infer']:
        sync_cmds.append(get_retrieve_for_proj(f'{projects[proj_id]}_{ix}'))
# the following was only used by accident
with open(Path(os.path.abspath('')) / 'output_sync.txt', 'w') as file:
    file.write('\n'.join(sync_cmds))
print(f'Stored {len(sync_cmds)} commands at output_sync.txt.')

Stored 24 commands at output_sync.txt.


## Feature and FIM extraction

This is how task feature extraction works. Note that full features comprise several GB and are not provided directly (also for licensing compatibility issues). The computed task distances are provided in the `cache` folder top-level.

In [23]:
updated_shrunk_task_list = [t.replace(' --shrink_train 800', '+shrink_train?800') for t in all_tasks_including_shrunk]

features_cmd = MMLJobDescription(prefix_req=def_reqs,
                                 config_options={'task_list': updated_shrunk_task_list, 'proj': 'pami2_features',
                                                 'mode.subroutines': ['feature'], 'augmentations': 'baseline256'},
                                 mode='emd')
fim_cmd = MMLJobDescription(prefix_req=def_reqs,
                            config_options={'task_list': updated_shrunk_task_list, 'proj': 'pami2_fims_recent',
                                            'mode.subroutines': ['tune', 'fim'], 'sampling.sample_num': 8000,
                                            'sampling.balanced': True, 'mode.fim.samples': 2000,
                                            'augmentations': 'baseline256', }, mode='fed')

In [ ]:
# the following demonstrates how to run these locally from within this notebook
# CAUTION: it produces a lot of logging output to the notebook - consider running these commands in the terminal as described below
from mml.interactive import SubprocessJobRunner

local_reqs = DefaultRequirements()
runner = SubprocessJobRunner()
for job in [features_cmd, fim_cmd]:
    job.prefix_req = local_reqs
    # runner.run(job)  # uncomment to run

In [24]:
# want to run in the terminal - follow here
local_reqs = DefaultRequirements()
for job in [features_cmd, fim_cmd]:
    job.prefix_req = local_reqs
features_cmd.render()  # paste the output into terminal (remove surrounding quotes) takes ~20 minutes

"mml emd task_list=['lapgyn4_anatomical_structures','lapgyn4_surgical_actions','lapgyn4_instrument_count','lapgyn4_anatomical_actions','sklin2_skin_lesions','identify_nbi_infframes','laryngeal_tissues','nerthus_bowel_cleansing_quality','stanford_dogs_image_categorization','svhn','caltech101_object_classification','caltech256_object_classification','cifar10_object_classification','cifar100_object_classification','mnist_digit_classification','emnist_digit_classification','hyperkvasir_anatomical-landmarks','hyperkvasir_pathological-findings','hyperkvasir_quality-of-mucosal-views','hyperkvasir_therapeutic-interventions','cholec80_grasper_presence','cholec80_bipolar_presence','cholec80_hook_presence','cholec80_scissors_presence','cholec80_clipper_presence','cholec80_irrigator_presence','cholec80_specimenbag_presence','derm7pt_skin_lesions','idle_action_recognition','barretts_esophagus_diagnosis','brain_tumor_classification','mednode_melanoma_classification','brain_tumor_type_classification'

In [25]:
fim_cmd.render()

"mml fed task_list=['lapgyn4_anatomical_structures','lapgyn4_surgical_actions','lapgyn4_instrument_count','lapgyn4_anatomical_actions','sklin2_skin_lesions','identify_nbi_infframes','laryngeal_tissues','nerthus_bowel_cleansing_quality','stanford_dogs_image_categorization','svhn','caltech101_object_classification','caltech256_object_classification','cifar10_object_classification','cifar100_object_classification','mnist_digit_classification','emnist_digit_classification','hyperkvasir_anatomical-landmarks','hyperkvasir_pathological-findings','hyperkvasir_quality-of-mucosal-views','hyperkvasir_therapeutic-interventions','cholec80_grasper_presence','cholec80_bipolar_presence','cholec80_hook_presence','cholec80_scissors_presence','cholec80_clipper_presence','cholec80_irrigator_presence','cholec80_specimenbag_presence','derm7pt_skin_lesions','idle_action_recognition','barretts_esophagus_diagnosis','brain_tumor_classification','mednode_melanoma_classification','brain_tumor_type_classification'